In [1]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from matplotlib import image
import os

In [2]:
# load images of size 112,92 

train_image_load = []
test_image_load = []

total_person = 40
train_size = 6
test_size = 4

# tracks train set and test set

for i in range(1,total_person+1):
    j = 1
    arr = os.listdir('s'+str(i)+'/')
    for _ in arr:
        if(j <= train_size):
            train_image_load.append((image.imread('s'+str(i)+'/'+_)).flatten())
        else:
            test_image_load.append((image.imread('s'+str(i)+'/'+_)).flatten())
        j+=1

In [3]:
def mean_face(Face):
    mn,p = Face.shape
    M = np.zeros((mn,1))
    for i in range(mn):
        for j in range(p):
            M[i] += Face[i][j]
        M[i] /= p
    return M

In [4]:
def surrogate_cov(X):
# Returns covariance of X
    rows,cols = X.shape 
    cov = np.zeros((cols,cols))
     
    # Covariances        
    for z in range(cols):
        for y in range(cols):
                cov[z][y] = np.dot(X[:,y].T,X[:,z])/rows                
    return cov 

In [5]:
def sub(X,M):
    res = X
    n,c = X.shape
    
    for i in range(n):
        for j in range(c):
            res[i][j] = res[i][j] - M[i]
    
    return res
    

### Step 1 Apply PCA

In [6]:

# step 1 generate face dataset (mn*p)
# 60% for training (6 images/folder)
Face_Db = np.array(train_image_load).T  
mn,p = Face_Db.shape

# step 2 calculate mean (mn*1)
M = mean_face(Face_Db)

# step 3 mean 0
delta = np.zeros((Face_Db.shape))
for i in range(mn):
    for j in range(p):
        delta[i][j] = Face_Db[i][j] - M[i]
        
# step 4 surrogate covariance calculation
cov = surrogate_cov(delta)   

# step 5 eigen values and eigen vectors (sorted decreasing order) of cov
eigenValues, eigenVectors = np.linalg.eig(cov)
idx = eigenValues.argsort()[::-1]   
eig_val = eigenValues[idx]
eig_vec = eigenVectors[:,idx]

# setting k
k = 5

# Psi
psi = eig_vec[:,0:k]

# step 7 Generate Eigen Faces
phi = np.dot(psi.T,delta.T)

PF = np.dot(phi,delta)


### Step 2 Divide data into classes

In [7]:
n = train_size
number_of_classes = p//n

### Step 3 Find means of each class and mean of Projected Faces

In [8]:
mean_each_class = np.zeros((k,number_of_classes))

old = 0
new = n
PF_c = []
for i in range(number_of_classes):
    # storing projected faces class wise
    PF_c.append(PF[:,old:new])
    mean_each_class[:,i] = mean_face(PF[:,old:new]).T
    old = new
    new = new + n

PF_c = np.array(PF_c)    
mean_PF = mean_face(PF)

In [9]:
mean_each_class.shape

(5, 40)

In [10]:
mean_PF.shape

(5, 1)

### Step 4: Within Class Scatter Matrix and Between Class Scatter Matrix

In [11]:
# Within Class Scatter Matrix
SW = np.zeros((k,k))
for i in range(number_of_classes):
    V_mu = sub(PF_c[i],mean_each_class[:,i])
    SW += (np.dot(V_mu,V_mu.T))
    
SW.shape

(5, 5)

In [12]:
# Between Class Scatter Matrix
SB = np.zeros((k,k))
for i in range(number_of_classes):
    M_i = np.reshape(mean_each_class[:,i],(1,mean_each_class[:,i].size)).T
    SB += (np.dot(sub(M_i,mean_PF),M_i.T))

SB.shape
    

(5, 5)

### Step 5: Criterion Function

In [14]:
J = (np.dot(np.linalg.inv(SW),SB))
J.shape

(5, 5)

### Step 6: Eigen Vector and Values of Criterion Function

In [16]:
J_eigenValues, J_eigenVectors = np.linalg.eig(J)
idx = J_eigenValues.argsort()[::-1]   
J_eigenValues = J_eigenValues[idx]
J_ev = J_eigenVectors[:,idx]

J_ev.shape

(5, 5)

### Step 7: Best Principle Component

In [17]:
m = 2

### Step 8: Feature$(W)_{k*m}$

In [18]:
W = J_ev[:,0:m]
W.shape

(5, 2)

### Step 9: Fisher Faces (FF) 

In [20]:
FF = np.dot(W.T,PF)
FF.shape

(2, 240)